In [1]:
import tensorflow as tf
print(tf.__version__)

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)
    

2.1.0


In [2]:
class  Residual(tf.keras.Model):
    def __init__(self,num_channels, use_1x1conv=False,strides=1,**kwargs):
        super(Residual,self).__init__(**kwargs)
        self.conv1 = tf.keras.layers.Conv2D(num_channels,padding='same',kernel_size=3,strides=strides)
        self.conv2 = tf.keras.layers.Conv2D(num_channels,kernel_size=3,padding='same')
        if use_1x1conv:
            self.conv3 = tf.keras.layers.Conv2D(num_channels,kernel_size=1,strides=strides)
        else:
            self.conv3 = None
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.bn2 = tf.keras.layers.BatchNormalization()
    

    def call(self,X):
        Y = tf.keras.activations.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        return tf.keras.activations.relu(Y+X)
        

In [3]:
net = tf.keras.models.Sequential(
    [tf.keras.layers.Conv2D(64,kernel_size=7,strides=2,padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D(pool_size=3,strides=2,padding='same')]
)

In [4]:
class ResnetBlock(tf.keras.layers.Layer):
    def __init__(self,num_channels,num_residuals,first_block=False,**kwargs):
        super(ResnetBlock,self).__init__(**kwargs)
        self.listLayers = []
        for i in range(num_residuals):
            if i==0 and not first_block:
                self.listLayers.append(Residual(num_channels, use_1x1conv=True, strides=2))
            else:
                self.listLayers.append(Residual(num_channels))
    
    def call(self,X):
        for layer in self.listLayers.layers:
            X = layer(X)
        return X

In [5]:
class ResNet(tf.keras.Model):
    def __init__(self,num_blocks,**kwargs):
        super(ResNet,self).__init__(**kwargs)
        self.conv = tf.keras.layers.Conv2D(64,kernel_size=7,strides=2,padding='same')
        self.bn = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.Activation('relu')
        self.mp = tf.keras.layers.MaxPool2D(pool_size=3,strides=2,padding='same')
        self.resnet_block1 = ResnetBlock(64,num_blocks[0],first_block=True)
        self.resnet_block2 = ResnetBlock(128,num_blocks[1])
        self.resnet_block3 = ResnetBlock(256,num_blocks[2])
        self.resnet_block4 = ResnetBlock(512,num_blocks[3])
        self.gap = tf.keras.layers.GlobalAvgPool2D()
        self.fc = tf.keras.layers.Dense(10,activation='softmax')
        
    def call(self,X):
        X = self.conv(X)
        X = self.bn(X)
        X = self.relu(X)
        X = self.mp(X)
        X = self.resnet_block1(X)
        X = self.resnet_block2(X)
        X = self.resnet_block3(X)
        X = self.resnet_block4(X)
        X = self.gap(X)
        X = self.fc(X)
        return X

In [6]:
mynet = ResNet([2,2,2,2])

In [7]:
X = tf.random.uniform(shape=(1,  224, 224 , 1))
for layer in mynet.layers:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

conv2d_1 output shape:	 (1, 112, 112, 64)
batch_normalization_1 output shape:	 (1, 112, 112, 64)
activation_1 output shape:	 (1, 112, 112, 64)
max_pooling2d_1 output shape:	 (1, 56, 56, 64)
resnet_block output shape:	 (1, 56, 56, 64)
resnet_block_1 output shape:	 (1, 28, 28, 128)
resnet_block_2 output shape:	 (1, 14, 14, 256)
resnet_block_3 output shape:	 (1, 7, 7, 512)
global_average_pooling2d output shape:	 (1, 512)
dense output shape:	 (1, 10)


In [8]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

mynet.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

history = mynet.fit(x_train, y_train,
                    batch_size=64,
                    epochs=5,
                    validation_split=0.2)
test_scores = mynet.evaluate(x_test, y_test, verbose=2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 29s 595us/sample - loss: 0.4796 - accuracy: 0.8319 - val_loss: 0.3784 - val_accuracy: 0.8611

In [9]:
mynet.evaluate(x_test,y_test)

10000/10000 [==============================] - 3s 270us/sample - loss: 0.2869 - accuracy: 0.8958

[0.2868509181499481, 0.8958]